In [1]:
import schedule
import time
import psycopg2
from psycopg2 import sql

def load_data():
    conn = psycopg2.connect(
        dbname="Nasa",
        user="postgres",
        password="SvetikAB",
        host="35.226.174.213",
        port="5432"
    )
    cur = conn.cursor()
    csv_file_path = 'db.csv'
    table_name = 'satellitedata'
    check_data_query = sql.SQL("SELECT EXISTS (SELECT 1 FROM {});").format(sql.Identifier(table_name))
    cur.execute(check_data_query)
    data_exists = cur.fetchone()[0]
    if data_exists:
        delete_query = sql.SQL("DELETE FROM {};").format(sql.Identifier(table_name))
        cur.execute(delete_query)
    sql_query = sql.SQL("""
        COPY {} FROM stdin WITH CSV HEADER;
    """).format(sql.Identifier(table_name))
    try:
        with open(csv_file_path, 'r') as f:
            cur.copy_expert(sql_query, f)
        conn.commit()
        if data_exists:
            print("Дані успішно перезаписано з CSV-файлу до бази даних.")
        else:
            print("Дані успішно завантажено з CSV-файлу до бази даних.")
    except psycopg2.Error as e:
        conn.rollback()
        print("Помилка під час завантаження даних:", e)
    finally:
        cur.close()
        conn.close()

# schedule.every().day.at("03:00").do(load_data)
schedule.every(10).seconds.do(load_data)  

while True:
    schedule.run_pending()
    time.sleep(5)



Дані успішно перезаписано з CSV-файлу до бази даних.


KeyboardInterrupt: 